In [1]:
from epinions_dataset import EpinionsDataset

# Load the dataset
dataset = EpinionsDataset("data/epinions.txt", 5000)

Processed 10000 lines
Processed 20000 lines
Processed 30000 lines
Processed 40000 lines
Processed 50000 lines
                                                user        item rating
0                 Minolta_QMS_PagePro_1250E_Printers      fgb59h    4.0
1  Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...    bucho_ky    2.0
2  Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...     redp944    4.0
3              pr-Durabrand_CD-85_Personal_CD_Player  spongebag7    4.0
4            pr-Sony_KV_36FS12__Standard_Televisions      pyros7    5.0
50000
Number of users: 38810
Number of items: 11525
